In [9]:
import gradio as gr
import os
from dotenv import load_dotenv
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import torch
import numpy as np
import scipy.io.wavfile
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import time

# GPU 사용 여부 확인 및 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125", temperature=0)

# 음악 설명 생성
def generate_response(mood, genre, elements, tempo):
    description = f"현재 기분: {mood}, 선호하는 음악 장르: {genre}, 음악 요소: {elements}, 분위기/속도: {tempo}"
    full_prompt = f"""
    당신은 ADHD 환자를 위한 음악 설명을 작성하는 어시스턴트입니다. 
    사용자의 입력을 바탕으로 음악 설명을 작성해야 합니다.
    사용자의 기분, 상황, 선호 장르, 분위기/속도에 맞춰 집중력을 향상시키고 스트레스를 감소시키는 음악의 설명을 작성해주세요.
    음악 설명은 다음의 예시의 형식대로 작성해야 합니다. 

    ## 예시
    이 음악은 차분하고 느린 템포의 클래식 음악으로, 바쁜 일상 속에서 집중력을 높이고 스트레스를 줄이는 데 도움을 줄 수 있습니다. 
    피아노와 바이올린의 부드러운 선율이 조화를 이루며, 마음을 편안하게 해줍니다. 
    특히 바흐의 '골드베르크 변주곡'처럼 복잡하지 않으면서도 집중력을 향상시키는 음악이 이상적입니다. 
    이 음악을 들으면 중요한 프로젝트를 완성하는 동안 마음의 안정을 찾고, 차분하게 일에 몰두할 수 있을 것입니다.

    {description}
    """
    result = llm([HumanMessage(content=full_prompt)])
    
    if hasattr(result, 'content'):
        return result.content
    elif hasattr(result, 'text'):
        return result.text
    else:
        return "No text content found in the response."

# 음악 생성 모델을 위한 음악 설명 요약 및 번역
def music_features(response):
    summary_prompt = f"""
    요약: 이 음악 설명에서 음악의 특징적인 부분만을 30자 이내로 간략하게 서술해주세요. 
    음악을 통해 얻을 수 있는 효과는 제거하고, 음악적 특성만을 서술해주세요. 
    답변은 영어로 출력합니다. 
    아래의 형식처럼 음악의 특성만을 고려하여 작성하고, 한 줄로 출력해주세요.

    출력 형식:
    a catchy beat for a podcast intro
    
    예시: 
    1. a catchy beat for a podcast intro
    2. a funky house with 80s hip hop vibes
    3. a chill song with influences from lofi, chillstep and downtempo
    \n\n'{response}'"""
    # 요약 요청을 모델에 전달
    summary_result = llm([HumanMessage(content=summary_prompt)])
    return summary_result

# 음악 생성 모델 초기화
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

def generate_music(summary, directory="./", duration_seconds=30):
    # Musicgen 모델을 사용하여 음악 생성
    inputs = processor(text=[summary], padding=True, return_tensors="pt")
    audio_values = model.generate(**inputs, max_new_tokens=256)
    sampling_rate = model.config.audio_encoder.sampling_rate
    
    # 파일 저장
    output_filename = f"{directory}music.wav"
    scipy.io.wavfile.write(output_filename, rate=sampling_rate, data=audio_values[0, 0].numpy())
    
    return output_filename

# Gradio 인터페이스용 통합 함수
def gradio_music_therapy(mood, genre, elements, tempo, duration_seconds):
    response = generate_response(mood, genre, elements, tempo)
    summary = music_features(response).content
    music_file = generate_music(summary, duration_seconds=duration_seconds)
    return response, music_file

#예제
def print_like_dislike(x: gr.LikeData):
    print(x.index, x.value, x.liked)

def add_message(history, message):
    history.append((message, None))
    return history, gr.Textbox(value=None, interactive=False)

def bot(history):
    response = "**That's cool!**"
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history


def meal_planner(time_of_day, meal_options):
    return f"{time_of_day}에는 {' 그리고 '.join(meal_options)}을(를) 드세요."

# Gradio Blocks를 사용하여 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("# 성인 ADHD 진단 및 치료를 위한 상담 챗봇: ETA bot")
    gr.Markdown("#### ADHD를 위한 자가진단 및 치료 챗봇입니다. 자가진단을 통해 ADHD 결과를 분석하고, 병원 치료 혹은 음악 치료를 받을 수 있습니다.")
    with gr.Tabs():
        
        # ADHD 자가진단 챗봇 탭
        with gr.TabItem("ADHD 자가진단 챗봇"):
            chatbot = gr.Chatbot(
                [],
                elem_id="chatbot",
                bubble_full_width=False
            )

            chat_input = gr.Textbox(interactive=True, 
            placeholder="ADHD를 위한 자가진단 및 치료 챗봇입니다. 자가진단을 통해 ADHD 결과를 분석하고, 병원 치료 혹은 음악 치료를 받을 수 있습니다. 궁금한 점을 질문해주세요!", 
            show_label=False)

            chat_msg = chat_input.submit(add_message, [chatbot, chat_input], [chatbot, chat_input])
            bot_msg = chat_msg.then(bot, chatbot, chatbot, api_name="bot_response")
            bot_msg.then(lambda: gr.Textbox(interactive=True), None, [chat_input])

            chatbot.like(print_like_dislike, None, None)


        with gr.TabItem("ADHD 치료 음악 생성"):
            with gr.Row():
                with gr.Column(scale=1):
                    mood_input = gr.Textbox(label="현재 기분 또는 상황", placeholder="예: 행복, 스트레스 받음 등")
                    genre_input = gr.Textbox(label="선호하는 음악 장르", placeholder="예: 클래식, 재즈 등")
                    elements_input = gr.Textbox(label="음악에 반영되었으면 하는 요소", placeholder="예: 자연 소리, 특정 악기 등")
                    tempo_input = gr.Textbox(label="원하는 음악의 분위기나 속도", placeholder="예: 차분하고 느린, 에너지 넘치는 등")
                    duration_input = gr.Slider(minimum=1, maximum=120, step=1, label="음악 길이 (초 단위)", value=30)
                    generate_button = gr.Button("음악 생성")
                with gr.Column(scale=1):
                    music_description_output = gr.Textbox(label="생성된 음악 설명")
                    music_file_output = gr.Audio(label="생성된 음악 파일")
            generate_button.click(gradio_music_therapy, 
                                  inputs=[mood_input, genre_input, elements_input, tempo_input, duration_input], 
                                  outputs=[music_description_output, music_file_output])
        

demo.launch(share=True)

c:\Users\DESKTOP\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://e10774444da90ade0e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
